In [ ]:
### Run this cell to import the packages you will need to unpack the dataset
# File manipulation and IO (input/output)
import tensorflow as tf
import os
import pickle
import zipfile

# Import numerical and dataframe handling
import numpy as np
import scipy
import pandas as pd

# Data preprocessing
from PIL import Image
from sklearn.utils import shuffle

# Model scoring
from sklearn.metrics import confusion_matrix
from sklearn import metrics

# Import standard machine learning machinery
import tensorflow as tf

# Garbage collection (for saving RAM during training)
import gc

# Import plotting functionality
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.ticker import MultipleLocator
import matplotlib

import gc

2024-07-30 17:41:00.142309: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-30 17:41:01.155000: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-07-30 17:41:01.155122: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-07-30 17:41:01.155131: W tensorfl

In [ ]:
print("Num GPUs Available: ", tf.config.list_physical_devices('GPU'))

Num GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2024-07-30 17:41:11.471610: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-30 17:41:11.484431: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-30 17:41:11.484783: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [ ]:
path = "../isicdataset/ISIC-images/"

In [ ]:
df = pd.read_csv(f'{path}metadata.csv')
df.head()

,isic_id,attribution,copyright_license,acquisition_day,age_approx,anatom_site_general,benign_malignant,clin_size_long_diam_mm,concomitant_biopsy,dermoscopic_type,...,mel_class,mel_mitotic_index,mel_thick_mm,mel_type,mel_ulcer,melanocytic,nevus_type,patient_id,personal_hx_mm,sex
0,ISIC_0000000,Anonymous,CC-0,NaN,55,anterior torso,benign,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,female
1,ISIC_0000001,Anonymous,CC-0,NaN,30,anterior torso,benign,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,female
2,ISIC_0000002,Anonymous,CC-0,NaN,60,upper extremity,malignant,NaN,True,NaN,...,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,female
3,ISIC_0000003,Anonymous,CC-0,NaN,30,upper extremity,benign,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,male
4,ISIC_0000004,Anonymous,CC-0,NaN,80,posterior torso,malignant,NaN,True,NaN,...,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,male


In [ ]:
df = pd.get_dummies(df, columns=['diagnosis'], dtype="int")

In [ ]:
columns = ['diagnosis_actinic keratosis',
       'diagnosis_basal cell carcinoma', 'diagnosis_melanoma',
       'diagnosis_nevus', 'diagnosis_seborrheic keratosis',
       'diagnosis_solar lentigo', 'diagnosis_squamous cell carcinoma']
df["diagnosis"] = df[columns].values.tolist()
df.head()

,isic_id,attribution,copyright_license,acquisition_day,age_approx,anatom_site_general,benign_malignant,clin_size_long_diam_mm,concomitant_biopsy,dermoscopic_type,...,personal_hx_mm,sex,diagnosis_actinic keratosis,diagnosis_basal cell carcinoma,diagnosis_melanoma,diagnosis_nevus,diagnosis_seborrheic keratosis,diagnosis_solar lentigo,diagnosis_squamous cell carcinoma,diagnosis
0,ISIC_0000000,Anonymous,CC-0,NaN,55,anterior torso,benign,NaN,False,NaN,...,NaN,female,0,0,0,1,0,0,0,"[0, 0, 0, 1, 0, 0, 0]"
1,ISIC_0000001,Anonymous,CC-0,NaN,30,anterior torso,benign,NaN,False,NaN,...,NaN,female,0,0,0,1,0,0,0,"[0, 0, 0, 1, 0, 0, 0]"
2,ISIC_0000002,Anonymous,CC-0,NaN,60,upper extremity,malignant,NaN,True,NaN,...,NaN,female,0,0,1,0,0,0,0,"[0, 0, 1, 0, 0, 0, 0]"
3,ISIC_0000003,Anonymous,CC-0,NaN,30,upper extremity,benign,NaN,False,NaN,...,NaN,male,0,0,0,1,0,0,0,"[0, 0, 0, 1, 0, 0, 0]"
4,ISIC_0000004,Anonymous,CC-0,NaN,80,posterior torso,malignant,NaN,True,NaN,...,NaN,male,0,0,1,0,0,0,0,"[0, 0, 1, 0, 0, 0, 0]"


In [ ]:
list_ds = tf.data.Dataset.list_files(path + '*.jpg', shuffle=False)

NameError: name 'path' is not defined

KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
for x in list_ds.take(5):
    print(x.numpy())
print(list_ds)

b'../isicdataset/ISIC-images/ISIC_0000000.jpg'
b'../isicdataset/ISIC-images/ISIC_0000001.jpg'
b'../isicdataset/ISIC-images/ISIC_0000002.jpg'
b'../isicdataset/ISIC-images/ISIC_0000003.jpg'
b'../isicdataset/ISIC-images/ISIC_0000004.jpg'
<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>


In [ ]:
val_size = round(len(df) * 0.2)
train_ds = list_ds.skip(val_size)
val_ds = list_ds.take(val_size)

In [ ]:
keys = tf.constant(df['isic_id'].tolist())
values = tf.constant([" ".join(map(str, i)) for i in df['diagnosis'].tolist()])

# Use tf.lookup.StaticHashTable instead of DenseHashTable
initializer = tf.lookup.KeyValueTensorInitializer(keys, values, key_dtype=tf.string, value_dtype=tf.string)
table = tf.lookup.StaticHashTable(initializer, default_value="NA")
table.lookup(tf.constant(['ISIC_0000000'])).numpy()

array([b'0 0 0 1 0 0 0'], dtype=object)

In [ ]:
def process_path(file_path):
    img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(img, channels=3)
    fn = tf.strings.split(file_path, sep='/')[-1]
    fn = tf.strings.split(fn, sep='.')[0]
    label = table.lookup(fn)
    label = tf.strings.to_number(tf.strings.split(label), out_type=tf.dtypes.int32)
    return [img, label]

In [ ]:
# for x in list_ds.take(1):
#     img, label = process_path(x.numpy())

In [ ]:
train_ds = train_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
def configure_for_performance(ds):
  ds = ds.cache()
  ds = ds.shuffle(buffer_size=1000)
  ds = ds.batch(32)
  ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
  return ds

In [ ]:
train_ds = configure_for_performance(train_ds)
val_ds = configure_for_performance(val_ds)

# The Model

In [ ]:
# base_model = tf.keras.applications.VGG16(
#     include_top=False,
#     weights="imagenet",
#     input_tensor=None,
#     input_shape=None,
#     pooling=None,
#     # classes=2,
#     # classifier_activation="softmax",
# )
# base_model.trainable = False

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(None, None, 3)),
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.RandomFlip(),
    tf.keras.layers.RandomZoom((0.2, 0.5)),
    # tf.keras.layers.RandomCrop(256, 256),
    tf.keras.layers.CenterCrop(256, 256),
    # tf.keras.layers.Input(shape=(256, 256, 3)),
    # base_model,
    tf.keras.layers.Flatten(),
    # tf.keras.layers.Dense(256, activation='gelu'),
    tf.keras.layers.Dense(128, activation='gelu'),
    tf.keras.layers.Dense(7, activation='softmax'),
])

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.CategoricalCrossentropy(),
  metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, None, None, 3)     0         
                                                                 
 random_flip (RandomFlip)    (None, None, None, 3)     0         
                                                                 
 random_zoom (RandomZoom)    (None, None, None, 3)     0         
                                                                 
 center_crop (CenterCrop)    (None, 256, 256, 3)       0         
                                                                 
 flatten (Flatten)           (None, 196608)            0         
                                                                 
 dense (Dense)               (None, 128)               25165952  
                                                                 
 dense_1 (Dense)             (None, 7)                 9

# Model Training

In [ ]:
# Train model
def process_path(file_path):
    img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [256, 256])
    fn = tf.strings.split(file_path, sep='/')[-1]
    fn = tf.strings.split(fn, sep='.')[0]
    label = table.lookup(fn)
    label = tf.strings.to_number(tf.strings.split(label), out_type=tf.dtypes.int32)
    return [img, label]

# Redefine datasets
train_ds = train_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = configure_for_performance(train_ds)
val_ds = configure_for_performance(val_ds)

class garbage_collect_callback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    gc.collect()
    
# Train model
model.fit(
  train_ds,
  validation_data=val_ds,
  callbacks=[garbage_collect_callback()],
  epochs=10
)

Epoch 1/10
2024-07-30 11:49:36.519407: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 337 of 1000
2024-07-30 11:49:46.342579: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 648 of 1000
2024-07-30 11:49:56.627805: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 965 of 1000
2024-07-30 11:49:57.347116: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:417] Shuffle buffer filled.
630/662 [===========================>..] - ETA: 57s - loss: 5.9255 - accuracy: 0.62052024-07-30 12:08:47.213027: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at whole_file_read_ops.cc:114 : FAILED_PRECONDITION: ../isicdataset/ISIC-images/licenses; Is a directory


InvalidArgumentError: Graph execution error:

StringToNumberOp could not correctly convert string: NA
	 [[{{node StringToNumber}}]]
	 [[IteratorGetNext]] [Op:__inference_train_function_7443]

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d80955c4-ad7b-4eff-b971-b372a13f61b2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>